In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
import time
from PID import PID

In [ ]:
# -------------------------
# Parameters (tuned for 320x240 cam)
# -------------------------
camera_width  = 320
camera_height = 240

BASE_SPEED    = 0.22         # forward speed (0..1). Start conservative.
STEER_GAIN    = 0.8          # mixes steering into motors
MAX_CMD       = 0.6          # clamp for safety

# PID on normalized error ([-1,1])
# If your PID library expects process_value and has setpoint, we keep setpoint = 0 and feed normalized error.
# (If your PID expects current value vs setpoint, we just use error as "current" and setpoint=0.)
steering_pid = PID(Kp=0.9, Ki=0.0, Kd=0.12, setpoint=0.0)  # solid starting point

# ROI band (bottom of the image)
ROI_Y1_FRACTION = 0.60
ROI_Y2_FRACTION = 0.95

# Loss handling
LOST_MAX = 8  # frames to keep last steering if line lost

# -------------------------
# State
# -------------------------
state = "FOLLOW_LINE"
robot_stop = True
_lost_frames = 0
_last_err_norm = 0.0

def _clip(x, lo=-1.0, hi=1.0):
    return max(lo, min(hi, x))

def _set_motors(left, right):
    left  = _clip(left,  -MAX_CMD, MAX_CMD)
    right = _clip(right, -MAX_CMD, MAX_CMD)
    robot.set_motors(left, right)

def follow_black_line(frame):
    global state, robot_stop

    h, w, _ = frame.shape

    # ---- CHANGED: focus on a bottom band only (reduces noise, makes centroid stable)
    roi_y1 = int(h * 0.6)          # bottom 40% of the image
    roi = frame[roi_y1:h, :]

    # Enhance contrast
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])

    # Color masks (on ROI)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # More inclusive black detection
    mask_black1 = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 15, 3
    )
    _, mask_black2 = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY_INV)
    mask_black = cv2.bitwise_or(mask_black1, mask_black2)

    # Morphological cleanup
    kernel = np.ones((3,3), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)

    # Green-square turn detection (still works inside ROI)
    contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    turn_signal = None

    for cnt in contours:
        x, y, gw, gh = cv2.boundingRect(cnt)
        cx = x + gw // 2
        cy = y + gh // 2

        # Region just below the square
        below_y1 = min(mask_white.shape[0]-1, y + gh)
        below_y2 = min(mask_white.shape[0], y + gh + 10)
        strip_below = mask_white[below_y1:below_y2, x:x+gw]
        white_below = np.mean(strip_below) > 128 if strip_below.size > 0 else False

        # Side regions (same vertical band)
        side_y1 = y
        side_y2 = y + gh
        left_x1 = max(0, x - gw)
        left_x2 = x
        right_x1 = x + gw
        right_x2 = min(mask_black.shape[1], x + 2 * gw)

        left_region = mask_black[side_y1:side_y2, left_x1:left_x2]
        right_region = mask_black[side_y1:side_y2, right_x1:right_x2]

        black_left = np.mean(left_region) > 128 if left_region.size > 0 else False
        black_right = np.mean(right_region) > 128 if right_region.size > 0 else False

        if white_below:
            if black_right and not black_left:
                turn_signal = "LEFT"
            elif black_left and not black_right:
                turn_signal = "RIGHT"

        color = (0,255,0) if white_below else (0,0,255)
        cv2.rectangle(roi, (x, y), (x+gw, y+gh), color, 2)

    # Line centroid (on ROI only)
    M = cv2.moments(mask_black)
    line_found = M["m00"] > 1500  # <-- CHANGED: require some area to avoid noise
    if line_found:
        cx_line = int(M["m10"] / M["m00"])
        cv2.circle(roi, (cx_line, roi.shape[0]//2), 5, (0,0,255), -1)
        steering_pid.setpoint = w / 2  # compare to full-frame center (x is same)

    # State machine
    if state == "FOLLOW_LINE":
        if line_found:
            steering = steering_pid.update(cx_line)

            # ---- CHANGED: clamp steering/motors so they stay in safe range
            steering = float(np.clip(steering, -0.35, 0.35))

            left_motor  = float(np.clip(forward_speed + steering, -1.0, 1.0))
            right_motor = float(np.clip(forward_speed - steering, -1.0, 1.0))
        else:
            left_motor = right_motor = 0.0

        if turn_signal == "LEFT":
            state = "TURN_LEFT"
        elif turn_signal == "RIGHT":
            state = "TURN_RIGHT"

    elif state == "TURN_LEFT":
        robot.set_motors(-turn_speed, turn_speed)
        time.sleep(turn_duration)
        state = "FOLLOW_LINE"
        return

    elif state == "TURN_RIGHT":
        robot.set_motors(turn_speed, -turn_speed)
        time.sleep(turn_duration)
        state = "FOLLOW_LINE"
        return

    # Safety stop
    if robot_stop:
        left_motor = right_motor = 0.0

    robot.set_motors(left_motor, right_motor)

    # Debug overlay (put ROI overlay back into the full frame for display)
    debug_vis = cv2.addWeighted(roi, 0.8, cv2.cvtColor(mask_black, cv2.COLOR_GRAY2BGR), 0.3, 0)
    full_vis = frame.copy()
    full_vis[roi_y1:h, :] = debug_vis  # <-- CHANGED: keep widget size consistent
    line_mask_widget.value = bgr8_to_jpeg(full_vis)


In [ ]:
robot.stop()
camera.stop()